In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import numpy as np
import matplotlib.pyplot as plt
import time
import girth
from girth import rasch_conditional
from irt.data.rasch import generate_data, generate_data_positive_scores
from irt.algorithms.spectral_estimator import spectral_estimate
from irt.algorithms import spectral_estimator
from irt.algorithms import conditional_mle, joint_mle
from irt.algorithms import rasch_mml
from irt.algorithms import eigen_vector_method
from irt.evaluation import eval_utils

# import data (you supply this function)
from scipy.stats import norm
# my_data = import_data(filename)

# # Assume its dichotomous data with True -> 1 and False -> 0
# tagged_data = tag_missing_data(my_data, [0, 1])

In [3]:
def relative_betas_error(beta, betah):
    assert(len(beta) == len(betah))
    beta_norm = beta - np.mean(beta)
    betah_norm = betah - np.mean(betah)
    return np.linalg.norm(beta_norm - betah_norm)

def relative_z_error(z, zh):
    return np.linalg.norm(z - zh)/np.linalg.norm(z)

    

In [63]:


errors_arr = []
errors_ase_arr = []
errors_cmle_arr = []
errors_mmle_arr = []
errors_choppin_arr = []
errors_garner_arr = []
errors_saaty_arr = []
errors_pair_arr = []

time_arr = []
time_ase_arr = []
time_cmle_arr = []
time_mmle_arr = []
time_choppin_arr = []
time_garner_arr = []
time_saaty_arr = []
time_pair_arr = []

n_array = [100, 200, 500, 1000] #, 2500, 5000]
n_trials = 20
m = 20

expected_num_problems = 5

p = expected_num_problems/m
p = 1.0
test_var = 2
betas = np.random.normal(0, test_var, size=(m,))
student_var = 3

for n in n_array:
    # w_students = np.ones((n,))
    # p = np.sqrt(np.log(m)/n)
    thetas = np.random.normal(0, student_var, size=(n,))

    error_ase = []
    error_cmle = []
    error_mmle = []
    error_choppin = []
    error_garner = []
    error_saaty = []
    error_pair = []
    
    time_ase = []
    time_cmle = []
    time_mmle = []
    time_garner = []
    time_choppin = []
    time_saaty = []
    time_pair = []
    
    auc_mmle = []
    auc_ase = []
    
    for _ in range(n_trials):
        # Generate data
        data = generate_data(betas, thetas, p)

        # Accelerated spectral method
        start = time.time()
        lambd = 1./(m * np.log(m))
        est_ase = spectral_estimate(data, lambd=lambd, regularization="uniform")
        time_ase += [(time.time() - start)]
        error_ase += [relative_betas_error(betas, est_ase)]
        
        
        # CMLE
        
        # Choppin method
#         start = time.time()
#         # est_choppin = eigen_vector_method.choppin_method(data, return_beta=True)
#         # est_choppin = rasch_conditional(data)["Difficulty"]
#         est_choppin = conditional_mle.rasch_conditional_modified(data)
#         # print(est_choppin)
        
#         time_choppin += [(time.time() - start)]
#         error_choppin += [relative_betas_error(betas, est_choppin)]
                
        # Saaty's method
        start = time.time()
        # est_saaty = eigen_vector_method.saaty_method(data, return_beta=True)
        est_saaty = conditional_mle.rasch_conditional_original(data, max_iters=1000000, return_beta=True)
        time_saaty += [(time.time() - start)]
        error_saaty += [relative_betas_error(betas, est_saaty)]
        
        # Pairwise method
        # start = time.time()
        # est_pair = eigen_vector_method.conditional_pairwise(data, 0.1)
        # time_pair += [(time.time() - start)]
        # error_pair += [relative_betas_error(betas, est_pair)]        

    errors_ase_arr.append(error_ase)
    errors_cmle_arr.append(error_cmle)
    errors_mmle_arr.append(error_mmle)
    errors_choppin_arr.append(error_choppin)
    errors_garner_arr.append(error_garner)
    errors_saaty_arr.append(error_saaty)
    
    time_ase_arr.append(time_ase)
    time_cmle_arr.append(time_cmle)
    time_mmle_arr.append(time_mmle)
    time_choppin_arr.append(time_choppin)
    time_garner_arr.append(time_garner)
    time_saaty_arr.append(time_saaty)
    
    print(f"n={n}, m={m}, ASE={np.nanmean(error_ase)} ({np.nanmean(time_ase)}), CMLE={np.nanmean(error_cmle)} ({np.nanmean(time_cmle)}), MMLE={np.nanmean(error_mmle)} ({np.nanmean(time_mmle)}), " +
          f"Choppin={np.nanmean(error_choppin)} ({np.nanmean(time_choppin)}), Pair={np.nanmean(error_pair)} ({np.nanmean(time_pair)}), Saaty={np.nanmean(error_saaty)} ({np.nanmean(time_saaty)})")

n=100, m=20, ASE=1.8897741757630346 (0.0035391688346862794), CMLE=nan (nan), MMLE=nan (nan), Choppin=nan (nan), Pair=nan (nan), Saaty=1.4808395095169322 (0.31213884353637694)
n=200, m=20, ASE=1.2375338556542836 (0.006629884243011475), CMLE=nan (nan), MMLE=nan (nan), Choppin=nan (nan), Pair=nan (nan), Saaty=0.9917606038985861 (0.3644156098365784)
n=500, m=20, ASE=0.752267319467532 (0.006219041347503662), CMLE=nan (nan), MMLE=nan (nan), Choppin=nan (nan), Pair=nan (nan), Saaty=0.6085449201047346 (0.3702566146850586)
n=1000, m=20, ASE=0.524908851340695 (0.013058972358703614), CMLE=nan (nan), MMLE=nan (nan), Choppin=nan (nan), Pair=nan (nan), Saaty=0.44320500225428416 (0.4627893567085266)


In [51]:
betas = np.random.normal(0, 1, size=(10,))
thetas = np.random.normal(0, 1, size=(200,))
p = 0.5
data = generate_data(betas, thetas, p)


In [62]:

start = time.time()
est_cmle = conditional_mle.rasch_conditional_modified(data, verbose=False)
print(est_cmle, time.time() - start)
print(relative_betas_error(est_cmle, betas))


# start = time.time()
# est_cmle = conditional_mle.rasch_conditional_gradient(data, verbose=False)
# print(est_cmle, time.time() - start)
# print(relative_betas_error(est_cmle, betas))


start = time.time()
est_cmle_og = conditional_mle.rasch_conditional_original(data, max_iters=int(1e6), verbose=False)
print(est_cmle_og, time.time() - start)
print(relative_betas_error(est_cmle_og, betas))

[-0.15495205 -0.00724269 -1.29153982  0.72369513  0.46503433 -0.19279702
 -0.34724883  1.14108768 -0.51003487  0.17399814] 19.633915901184082
2.0150779748321432
[-0.15495049 -0.00724312 -1.29153977  0.7236956   0.4650336  -0.19279625
 -0.34724836  1.14108791 -0.51003592  0.17399678] 0.19191932678222656
2.0150779109535395
